In [2]:
import numpy as np
import pandas as pd
from datetime import datetime

today = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0).strftime("%Y%m%d")
today

'20250417'

## Processing Raw Test Cases

In [52]:
colorder = ['Device',"대분류","Section", "소분류", '사전조건', '테스트 항목', '기대결과']
df_tests = pd.read_csv('data/KOCOWA_3.5_test_cases.csv')[colorder].rename(columns={"Section": "Scenario"}).fillna("")

# Create one-hot encoded device columns
df_device_dummies = pd.get_dummies(df_tests["Device"], prefix="Device")

# Combine with the rest of the test case columns
df_tests_per_device = pd.concat([df_device_dummies, df_tests.drop(columns=["Device"])], axis=1)

df_tests_per_device = df_tests_per_device.rename(columns={'Device_AOS': "Android Mobile", 'Device_AndroidTV': "Android TV", 'Device_AppleTV': "Apple TV",
       'Device_Connected TV': "Connected TV", 'Device_FireTV': "Fire TV", 'Device_Mobile': "Mobile", 'Device_ROKU': "Roku",
       'Device_Tablet': "Tablet", 'Device_WEB': "Web", 'Device_iOS': "Apple Mobile"})

for index, row in df_tests_per_device.iterrows():
       if row.Mobile == True:
              df_tests_per_device.at[index, "Android Mobile"] = True
              df_tests_per_device.at[index, "Apple Mobile"] = True
       scenario_items = row.Scenario.strip().split(">")
       df_tests_per_device.at[index, "소분류"] = scenario_items[-1]

df_tests_per_device = df_tests_per_device.drop(columns=["Mobile", "Tablet"])

df_tests_per_device.sort_values(by=["대분류","Scenario","소분류","테스트 항목", "기대결과"])


,Android Mobile,Android TV,Apple TV,Connected TV,Fire TV,Roku,Web,Apple Mobile,대분류,Scenario,소분류,사전조건,테스트 항목,기대결과
580,False,False,False,False,False,False,True,False,Category List Area,Category List Area,Category List Area,1. PLAN Page 진입,Title Text 확인,"""Subscription"" text 출력된다"
588,False,False,False,False,False,False,True,False,Category List Area,Category List Area>Basic Plan Option,Basic Plan Option,1. PLAN Page 진입,Bullet point 확인,"Benefit text 앞 ""●"" 형태의 글머리 기호 노출된다"
593,False,False,False,False,False,False,True,False,Category List Area,Category List Area>Basic Plan Option>Annual Su...,Annual Subscription,1. PLAN Page 진입,item Hover 시 UI 확인,item 흰색으로 강조되어 노출된다
596,False,False,False,False,False,False,True,False,Category List Area,Category List Area>Basic Plan Option>Annual Su...,Annual Subscription,1. PLAN Page 진입\n2. Basic Montlhy Subscription...,item 결제 시 UI 확인,해당 Subscription item Dimmed 처리되어 노출된다
595,False,False,False,False,False,False,True,False,Category List Area,Category List Area>Basic Plan Option>Annual Su...,Annual Subscription,1. PLAN Page 진입,item 선택 시 동작 확인,Checkout Page 진입된다
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,True,False,False,False,False,False,False,True,Welcome to KOCOWA Page,Welcome to KOCOWA Page>Body,Body,1. Sign Up Page 진입\n2. KOCOWA Account Data 입력\...,Next Button 확인,"""Next"" Button 노출된다"
950,True,False,False,False,False,False,False,True,Welcome to KOCOWA Page,Welcome to KOCOWA Page>Body,Body,1. Sign Up Page 진입\n2. KOCOWA Account Data 입력\...,User Email address 확인,User Email Address 출력된다
951,True,False,False,False,False,False,False,True,Welcome to KOCOWA Page,Welcome to KOCOWA Page>Body,Body,1. Sign Up Page 진입\n2. KOCOWA Account Data 입력\...,sub text 확인,"""An authentication email will arrive to your e..."
949,True,False,False,False,False,False,False,True,Welcome to KOCOWA Page,Welcome to KOCOWA Page>Body,Body,1. Sign Up Page 진입\n2. KOCOWA Account Data 입력\...,title text 확인,"""Completed to create your account"" text 출력된다"


## Processing Billing Gateway Validation Test Cases to Append

### Remove Country identifier in Scenario

In [53]:
colorder = ["대분류","Section", "소분류", '사전조건', '테스트 항목', '기대결과']
df_tests_billing = pd.read_csv('data/gateway-validation-tc.csv')[colorder].rename(columns={"Section": "Scenario"}).fillna("")
country_list = ["US", "Argentina", "Spain", "United Kingdom", "Australia", "Brazil"]
billing_categories = ['Cancel PLAN', 'Checkout Page','Checkout Page\n(Coupon)','PLAN Change', 'PLAN Downgrade', 'PLAN Upgrade','Premium Pass','Renewal Subscription','Resume PLAN']
df_tests_billing = df_tests_billing[df_tests_billing["대분류"].isin(billing_categories)]
devices = ['Android Mobile', 'Android TV', 'Apple TV', 'Connected TV', 'Fire TV','Roku', 'Web', 'Apple Mobile']

for index, row in df_tests_billing.iterrows():
    scenario_items = row.Scenario.strip().split(">")
    cleaned_items = []
    
    for item in scenario_items:
        item = item.strip()
        if item not in country_list:
            cleaned_items.append(item)
    
    if len(cleaned_items) > 1:
        new_scenario = ">".join(cleaned_items)
    elif cleaned_items:
        new_scenario = cleaned_items[0]
        
    if row.대분류 in ['PLAN Downgrade', 'PLAN Upgrade']:
        df_tests_billing.at[index, "대분류"] = "PLAN Change"
    
    for device in devices:
        df_tests_billing.at[index, device] = True
    
    df_tests_billing.at[index, "소분류"] = cleaned_items[-1]
    df_tests_billing.at[index, "Scenario"] = new_scenario

df_tests_billing[['Android Mobile', 'Android TV', 'Apple TV', 'Connected TV', 'Fire TV','Roku', 'Web', 'Apple Mobile', '대분류', 'Scenario', '소분류', '사전조건','테스트 항목', '기대결과']].sort_values(["대분류"])



,Android Mobile,Android TV,Apple TV,Connected TV,Fire TV,Roku,Web,Apple Mobile,대분류,Scenario,소분류,사전조건,테스트 항목,기대결과
2057,True,True,True,True,True,True,True,True,Cancel PLAN,Cancel PLAN>Premium Monthly,Premium Monthly,1. PLAN Page 진입\n2. Premium Monthly item 결제\n3...,Cancel Button 선택 시 기능 확인,구독 취소된다
2060,True,True,True,True,True,True,True,True,Cancel PLAN,Cancel PLAN>Premium Monthly,Premium Monthly,1. PLAN Page 진입\n2. Premium Monthly item 결제\n3...,Cancel PLAN 적용 시 동작 확인,Resume Button 노출된다
2061,True,True,True,True,True,True,True,True,Cancel PLAN,Cancel PLAN>Premium Annual,Premium Annual,1. PLAN Page 진입\n2. Premium Annual item 결제\n3....,Cancel Button 노출 확인,Cancel Button 노출된다
2062,True,True,True,True,True,True,True,True,Cancel PLAN,Cancel PLAN>Premium Annual,Premium Annual,1. PLAN Page 진입\n2. Premium Annual item 결제\n3....,Cancel Button 선택 시 기능 확인,구독 취소된다
2063,True,True,True,True,True,True,True,True,Cancel PLAN,Cancel PLAN>Premium Annual,Premium Annual,1. PLAN Page 진입\n2. Premium Annual item 결제\n3....,구독 유효 text 출력 확인,Valid Date : text 출력된다
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2091,True,True,True,True,True,True,True,True,Resume PLAN,Resume PLAN>Premium Monthly,Premium Monthly,1. PLAN Page 진입\n2. Premium Monthly item 결제\n3...,구독 갱신 text 출력 확인,Next Billing Date : text 출력된다
2092,True,True,True,True,True,True,True,True,Resume PLAN,Resume PLAN>Premium Monthly,Premium Monthly,1. PLAN Page 진입\n2. Premium Monthly item 결제\n3...,구독 갱신 일자 노출 확인,"[MM/DD/YYYY, HH:MM] [AM/PM] [대륙/도시] 노출된다"
2093,True,True,True,True,True,True,True,True,Resume PLAN,Resume PLAN>Premium Annual,Premium Annual,1. PLAN Page 진입\n2. Premium Annual item 결제\n3....,Resume Button 선택 시 기능 확인,구독 재개된다
2096,True,True,True,True,True,True,True,True,Resume PLAN,Resume PLAN>Premium Monthly,Premium Monthly,1. PLAN Page 진입\n2. Premium Monthly item 결제\n3...,Resume Button 선택 시 기능 확인,구독 재개된다


### Combine the two

In [67]:
df_tests_per_device = pd.concat([df_tests_per_device, df_tests_billing], axis=0)

df_tests_per_device.sort_values(by=["대분류"])

,Android Mobile,Android TV,Apple TV,Connected TV,Fire TV,Roku,Web,Apple Mobile,대분류,Scenario,소분류,사전조건,테스트 항목,기대결과
1191,True,True,True,True,True,True,True,True,Cancel PLAN,Cancel PLAN,Cancel PLAN,1. PLAN Page 진입\n2. Basic Monthly item 결제\n3. ...,Resume Button 선택 시 기능 확인,구독 재개된다
2046,True,True,True,True,True,True,True,True,Cancel PLAN,Cancel PLAN>Premium Monthly,Premium Monthly,1. PLAN Page 진입\n2. Premium Monthly item 결제\n3...,Cancel Button 노출 확인,Cancel Button 노출된다
2045,True,True,True,True,True,True,True,True,Cancel PLAN,Cancel PLAN>Premium Annual,Premium Annual,1. PLAN Page 진입\n2. Premium Annual item 결제\n3....,Cancel PLAN 적용 시 동작 확인,Resume Button 노출된다
2044,True,True,True,True,True,True,True,True,Cancel PLAN,Cancel PLAN>Premium Annual,Premium Annual,1. PLAN Page 진입\n2. Premium Annual item 결제\n3....,구독 유효 일자 노출 확인,"[MM/DD/YYYY, HH:MM] [AM/PM] [대륙/도시] 노출된다"
2043,True,True,True,True,True,True,True,True,Cancel PLAN,Cancel PLAN>Premium Annual,Premium Annual,1. PLAN Page 진입\n2. Premium Annual item 결제\n3....,구독 유효 text 출력 확인,Valid Date : text 출력된다
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,True,False,False,False,False,False,False,True,Welcome to KOCOWA Page,Welcome to KOCOWA Page>Header,Header,1. Sign Up Page 진입\n2. KOCOWA Account Data 입력\...,title text 확인,"""Welcome to KOCOWA"" text 출력된다"
949,True,False,False,False,False,False,False,True,Welcome to KOCOWA Page,Welcome to KOCOWA Page>Body,Body,1. Sign Up Page 진입\n2. KOCOWA Account Data 입력\...,title text 확인,"""Completed to create your account"" text 출력된다"
950,True,False,False,False,False,False,False,True,Welcome to KOCOWA Page,Welcome to KOCOWA Page>Body,Body,1. Sign Up Page 진입\n2. KOCOWA Account Data 입력\...,User Email address 확인,User Email Address 출력된다
953,True,False,False,False,False,False,False,True,Welcome to KOCOWA Page,Welcome to KOCOWA Page>Body,Body,1. Sign Up Page 진입\n2. KOCOWA Account Data 입력\...,Next Button 기능 확인,Multi-Profile Main Page 진입된다


## Pivot Data to Use Device Columns as Boolean Flags

In [66]:
test_case_keys = ["대분류","소분류","테스트 항목"]

device_columns = ['Android Mobile', "Apple Mobile", 'Android TV', 'Apple TV', 'Connected TV', 'Fire TV', 'Roku', 'Web']

df_tests_grouped = df_tests_per_device.groupby(test_case_keys)[device_columns].max().reset_index()
df_tests_grouped = df_tests_grouped.sort_values(by=test_case_keys).reset_index(drop=True)
print(df_tests_grouped.소분류.unique())
# print(df_tests_per_device.Scenario.unique())
# print(len(list(df_tests_grouped.index)))

df_tests_grouped

['Cancel PLAN' 'Premium Annual' 'Premium Annual Single Pass'
 'Premium Month Single Pass' 'Premium Monthly' 'Annual Subscription'
 'Basic Plan Option' 'Bottom Area' 'Category List Area' 'Header Area'
 'Monthly Subscription' 'Premium Plan Option' 'Basic Annual'
 'Basic Annual Single Pass' 'Basic Annuals' 'Basic Month Single Pass'
 'Basic Monthly' 'Billing Information Area' 'Checkout Page'
 'Payment Summary Area' 'Card Change' 'Casting Action'
 'Casting Player Screen' 'Player UI' 'Curation Area'
 'Hovered Poster image' 'Functional Area' 'IME' 'Validation'
 'input box Area' 'Email Input Area' 'Forgot Password text Button'
 'Password Input Area' 'Sign In Button' 'Sign Up text Button'
 'Name Input Area' 'Next Button' 'Basic Annual to Basic Monthly'
 'Basic Annual to Premium Annual'
 'Basic Annual to Premium Annual\n(Credit)'
 'Basic Annual to Premium Monthly' 'Basic Monthly to Basic Annual'
 'Basic Monthly to Basic Annual\n(Credit)'
 'Basic Monthly to Premium Annual'
 'Basic Monthly to Prem

,대분류,소분류,테스트 항목,Android Mobile,Apple Mobile,Android TV,Apple TV,Connected TV,Fire TV,Roku,Web
0,Cancel PLAN,Cancel PLAN,Cancel Button 노출 확인,True,True,True,True,True,True,True,True
1,Cancel PLAN,Cancel PLAN,Cancel Button 선택 시 기능 확인,True,True,True,True,True,True,True,True
2,Cancel PLAN,Cancel PLAN,Resume Button 노출 여부 확인,True,True,True,True,True,True,True,True
3,Cancel PLAN,Cancel PLAN,Resume Button 선택 시 기능 확인,True,True,True,True,True,True,True,True
4,Cancel PLAN,Cancel PLAN,WEB 상품 Cancel PLAN 적용 시 동작 확인,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...
1227,Welcome to KOCOWA Page,Body,Next Button 확인,True,True,False,False,False,False,False,False
1228,Welcome to KOCOWA Page,Body,User Email address 확인,True,True,False,False,False,False,False,False
1229,Welcome to KOCOWA Page,Body,sub text 확인,True,True,False,False,False,False,False,False
1230,Welcome to KOCOWA Page,Body,title text 확인,True,True,False,False,False,False,False,False


In [68]:
df_tests_grouped[df_tests_grouped["소분류"].isin(["Upgrade", "Downgrade"])]

,대분류,소분류,테스트 항목,Android Mobile,Apple Mobile,Android TV,Apple TV,Connected TV,Fire TV,Roku,Web
286,PLAN Change,Downgrade,Downgrade item 구독 시작 일자 노출 확인,True,True,True,True,True,True,True,True
287,PLAN Change,Downgrade,Downgrade item 노출 확인,True,True,True,True,True,True,True,True
288,PLAN Change,Downgrade,Expires on text 출력 확인,True,True,True,True,True,True,True,True
289,PLAN Change,Downgrade,Plan Update Email 수신 확인,True,True,True,True,True,True,True,True
290,PLAN Change,Downgrade,Start on text 출력 확인,True,True,True,True,True,True,True,True
291,PLAN Change,Downgrade,결제 Email 수신 확인,True,True,True,True,True,True,True,True
292,PLAN Change,Downgrade,기존 Subscribe item 노출 확인,True,True,True,True,True,True,True,True
293,PLAN Change,Downgrade,기존 item 구독 시작 일자 노출 확인,True,True,True,True,True,True,True,True
317,PLAN Change,Upgrade,Plan Update Email 수신 확인,True,True,True,True,True,True,True,True
318,PLAN Change,Upgrade,Receipt Email 확인,True,True,True,True,True,True,True,True


### Export: {date}_processed_test_cases.csv

In [64]:
df_tests_grouped.to_csv(f"data/{today}_processed_test_cases.csv", index=False)